# Embeddings con LSTM



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter
import random
# from itertools import chain
# from pprint import pprint
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [2]:
cd ../helpers/

C:\Users\Usuario.000\Documents\Facultad\Git\2020-2\APIT-2020-2\ProyectoFinal\helpers


In [3]:
import boletines

In [4]:
cd ../Embeddings/

C:\Users\Usuario.000\Documents\Facultad\Git\2020-2\APIT-2020-2\ProyectoFinal\Embeddings


## Definición de constantes

In [5]:
# Número de GPUs disponibles. Usar 0 para modo CPU.
ngpu = 1

# Semilla a usar en los generadores de números aleatorios
SEED = 42069
# SEED = random.randint(1, 10000) # En caso de requerir más resultados
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

print("Random Seed: ", SEED)

Random Seed:  42069


Selecciono el tipo de dispositivo a utilizar (gpu o cpu)

In [6]:
# Decide si queremos correr en gpu o cpu
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
device

device(type='cuda', index=0)

## Obtener corpus

In [7]:
uri = open('./../mongo_uri.txt', 'r', encoding='utf-8').read()

In [9]:
%%time
dict_sentences = boletines.sentences(uri)

Wall time: 3.05 s


Paso los tokens a minúsculas para reducir el tamaño del vocabulario

In [13]:
sentences = [[w.lower() for w in sent] for sent in dict_sentences['sentences']]
print(f'El corpus consta de {dict_sentences["count"]} enunciados.')
print(f'Tamaño mínimo: {dict_sentences["min"]} tokens.')
print(f'Tamaño máximo: {dict_sentences["max"]} tokens.')
print(f'Tamaño promedio: {dict_sentences["avg"]} tokens.')
print(f'Desviación estándar: {dict_sentences["std"]} tokens.')

El corpus consta de 6564 enunciados.
Tamaño mínimo: 5 tokens.
Tamaño máximo: 129 tokens.
Tamaño promedio: 38.22943327239488 tokens.
Desviación estándar: 18.479413427637674 tokens.


Imprimo algunas oraciones de muestra

In [17]:
step = max(dict_sentences["count"]//5, 1)
print('\n\n'.join(
    [ f"{i+1}.- {' '.join(sent)}" for i, sent in enumerate(sentences[::step]) ]
))

1.- asegura amlo que en campaña todo será amor y paz , que los otros candidatos se ahorren sus provocaciones .

2.- a la pregunta de los reporteros de cómo es su preparación en el debate , lópez_obrador respondió que aquí se está preparando a el conceder una entrevista a los representantes de los medios de comunicación .

3.- frente_a quienes plantean dividir a la nación , el candidato de el pri , pvem y nueva_alianza prometió unidad y seguridad jurídica para que se invierta y se generen empleos .

4.- a la pregunta de los reporteros qué si tiene algún comentario sobre lo que dijo meade de acusar de secuestradora a nestora_salgado , lópez_obrador mencionó que se dicen muchas mentiras y “ ella fue calumniada , ella es una dirigente social , está luchando , porque haya paz y tranquilidad ” .

5.- así lo señaló arturo_zamora , líder de la confederación_nacional_de_organizaciones_populares , tras apuntar que el encuentro entre abanderados presidenciales , a celebrar se en mérida , dejará e

## Indexar Corpus

Cuento los tokens en todas las oraciones e imprimo algunos de ellos.

In [18]:
%%time
freq = sum([Counter(sent) for sent in sentences], Counter())

Wall time: 27.6 s


In [19]:
n_tipos = len(freq.keys())
print(f'Número de tipos: {n_tipos}')
print(freq.most_common()[::max(n_tipos//25, 1)])

Número de tipos: 13454
[(',', 19107), ('mujer', 46), ('grave', 21), ('mala', 13), ('llegará', 9), ('profesional', 7), ('respecto_de', 5), ('pozos', 4), ('nuño', 4), ('íntegro', 3), ('liberal', 3), ('desempeño', 2), ('presidencia_de_la_república_actúa', 2), ('martha_tagle', 2), ('protestar', 2), ('secundamos', 1), ('aéreas', 1), ('engañe', 1), ('mediático', 1), ('pondremos', 1), ('coordinaciones', 1), ('lamentaron', 1), ('esforzaron', 1), ('construí', 1), ('innovaciones', 1), ('decepcionarán', 1)]


Agrego un diccionario para pasar de palabra a índice numérico

In [20]:
w_to_ix = {
    w : ix
    for ix, (w, freq) in enumerate(freq.most_common())
    if freq > 1 # No toma en cuenta los hapax
}

In [21]:
vocab_size = len(w_to_ix)
vocab_size

7609

Agrego símbolos de inicio y fin de cadena, así como el token <UNK\> para palabras no vistas

In [22]:
BOS = '<BOS>'
EOS = '<EOS>'
UNK = '<UNK>'

ixBOS = vocab_size
ixEOS = vocab_size + 1
ixUNK = vocab_size + 2

In [24]:
w_to_ix[BOS] = ixBOS
w_to_ix[EOS] = ixEOS
w_to_ix[UNK] = ixUNK

Calculo el nuevo tamaño del vocabulario después de agregar 3 tokens

In [25]:
vocab_size = len(w_to_ix)
vocab_size

7612

Creo el diccionario inverso, para convertir de índices a palabras

In [26]:
ix_to_w = [ w for w, ix in w_to_ix.items() ]

Guardo las etiquetas en un archivo txt

In [48]:
with open('word_labels_freeling.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(ix_to_w))

Indexo todo el corpus

In [27]:
def w_to_index_unk(w):
    """
    Le asigna el token UNK a palabras que no aparezcan en el corpus
    """
    try:
        return w_to_ix[w] 
    except KeyError:
        return ixUNK

In [28]:
sentences_ix = [
    # Le agrego el inicio y fin de caracter a los enunciados
    [ixBOS] + [ w_to_index_unk(w) for w in sent ] + [ixEOS] 
    for sent in sentences
]

X = [ np.asarray(sent[:-1]) for sent in sentences_ix ]
Y = [ np.asarray(sent[1:]) for sent in sentences_ix ] 

print('Ejemplos X')
print(X[:3])
print('Ejemplos Y')
print(Y[:3])

Ejemplos X
[array([7609,  634,  258,    2,    8,   54,   57,   68,  381,    7,   65,
          0,    2,    9,  213,  110,   10, 5497,   35, 4308,    6]), array([7609, 1089,   42,  242,    2,   11, 1818,  103,    3,   84,    1,
        163,  108,   10, 3600,   16,  288, 1405,    6]), array([7609,  667, 2261,    2,  316,   10, 2496,    7,   20,  298, 1264,
          8,  792,    6])]
Ejemplos Y
[array([ 634,  258,    2,    8,   54,   57,   68,  381,    7,   65,    0,
          2,    9,  213,  110,   10, 5497,   35, 4308,    6, 7610]), array([1089,   42,  242,    2,   11, 1818,  103,    3,   84,    1,  163,
        108,   10, 3600,   16,  288, 1405,    6, 7610]), array([ 667, 2261,    2,  316,   10, 2496,    7,   20,  298, 1264,    8,
        792,    6, 7610])]


In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30)

In [30]:
print(len(X_train))
print(len(X_test))

4594
1970


Mando los vectores de entrada y salida a tensores en gpu

In [31]:
def to_pytorch_tensor(list_of_lists):
    return [
        torch.from_numpy(l).long().to(device)
        for l in list_of_lists
    ]

In [32]:
X_train = to_pytorch_tensor(X_train)
Y_train = to_pytorch_tensor(Y_train)

X_test = to_pytorch_tensor(X_test)
Y_test = to_pytorch_tensor(Y_test)

## Modelo

### 1. Capa de embedding

### 2. Capa oculta

### 3. Capa de salida

Defino las variables para la red neuronal

In [33]:
# Dimensión de entrada (one-hot), tamaño del vocabulario
D_in = vocab_size

# Dimensión de la capa de embedding
D_emb = 64 # 32

# Dimensión de la capa lstm
D_lstm = 32 # 16

# Dimensión de la capa de salida
D_out = D_in

# Épocas de entrenamiento
num_epochs = 350

# Learning rate
lr =  0.1

In [34]:
class Model(nn.Module):
    def __init__(self, ngpu, D_in, D_emb, D_lstm, D_out):
        super(Model, self).__init__()
        self.ngpu = ngpu
        self.embedding = nn.Embedding(num_embeddings=D_in, embedding_dim=D_emb)#, padding_idx=0)
        self.lstm = nn.LSTM(input_size=D_emb, hidden_size=D_lstm) #, bias=True)#, batch_first=True)
        self.linear = nn.Linear(in_features=D_lstm, out_features=D_out) #, bias=True)

    def forward(self, sentence):
        T = len(sentence)
        embeddings = self.embedding(sentence).view(T, 1, -1)
        lstm_out, (ht, ct) = self.lstm(embeddings)
        lstm_out = lstm_out.view(T, -1)
        preact_out = self.linear(lstm_out).view(T, -1)
        
        return F.log_softmax(preact_out, dim=1)
    
    def pred(self, sentence):
        with torch.no_grad():
            out = self.forward(sentence)
            pred = torch.argmax(out, dim=1)
        return pred

In [35]:
model = Model(ngpu, D_in, D_emb, D_lstm, D_out).to(device)
model

Model(
  (embedding): Embedding(7612, 64)
  (lstm): LSTM(64, 32)
  (linear): Linear(in_features=32, out_features=7612, bias=True)
)

Verisimilitud logarítmica negativa como función de coste y SGD como optimizador

In [36]:
criterion = nn.NLLLoss()
criterion.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

### Entrenamiento

In [38]:
%%time
for epoch in tqdm(range(num_epochs)):
    epoch_loss = 0
    for x, y in zip(X_train, Y_train):
        if len(x) == 0:
            tqdm.write('Sentencia vacía')
            continue
        # Limpiamos gradientes acumulados
        model.zero_grad()

        # Forward
        out = model(x)
     
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    if epoch%20 == 0:
        tqdm.write(f'Coste después de {epoch+1} épocas: {epoch_loss}')

Coste después de 1 épocas: 27134.14709877968
Coste después de 21 épocas: 20425.387298464775
Coste después de 41 épocas: 19120.266835570335
Coste después de 61 épocas: 18274.351877570152
Coste después de 81 épocas: 17647.85674071312
Coste después de 101 épocas: 17136.896490573883
Coste después de 121 épocas: 16723.770038604736
Coste después de 141 épocas: 16384.72590970993
Coste después de 161 épocas: 16087.5071747303
Coste después de 181 épocas: 15829.571817457676
Coste después de 201 épocas: 15635.830556929111
Coste después de 221 épocas: 15437.284291267395
Coste después de 241 épocas: 15280.804988384247
Coste después de 261 épocas: 15113.961046397686
Coste después de 281 épocas: 15020.560606181622
Coste después de 301 épocas: 14998.55774563551
Coste después de 321 épocas: 14915.654001414776
Coste después de 341 épocas: 14782.115661382675

Wall time: 2h 20min 22s


In [42]:
def print_ix_sentence(sentence):
    print(' '.join(ix_to_w[ix] for ix in sentence.data))

In [43]:
# See what the scores are after training
with torch.no_grad():
    for sentence, y in zip(X_test[:10], Y_test[:10]):
        print('===================================================')
        print('- Probando enunciado:')
        print_ix_sentence(sentence)
        print()
        
        print('- Real:')
        print_ix_sentence(y)
        print()
        
        prediccion = model.pred(sentence)
        print('- Predicción:')
        print_ix_sentence(prediccion)
        print('\n')
        

- Probando enunciado:
<BOS> “ lo que nosotros vamos a hacer es prevención y enfrentar el problema por decisión , con una nueva estrategia .

- Real:
“ lo que nosotros vamos a hacer es prevención y enfrentar el problema por decisión , con una nueva estrategia . <EOS>

- Predicción:
“ vamos que se es a ganar una para de a la gobierno de la de la la campaña estrategia de <EOS>


- Probando enunciado:
<BOS> entre los empresarios que se dieron cita , destacaron <UNK> , <UNK> , <UNK> , <UNK> , <UNK> , <UNK> , <UNK> y <UNK> , <UNK> , <UNK> , <UNK> y <UNK> , entre otros .

- Real:
entre los empresarios que se dieron cita , destacaron <UNK> , <UNK> , <UNK> , <UNK> , <UNK> , <UNK> , <UNK> y <UNK> , <UNK> , <UNK> , <UNK> y <UNK> , entre otros . <EOS>

- Predicción:
“ los candidatos , no <UNK> los , que se la que en que a que a que a que a que y <UNK> , porque a que a que a <UNK> , porque todos condiciones <EOS>


- Probando enunciado:
<BOS> expuso que no vivirá ni en los_pinos , ni en una mansión

In [46]:
torch.save(
    model.state_dict(), 
    f'./modelsaves/model-freeling-emb_{D_emb}-lstm_{D_lstm}-seed_{SEED}-epochs_{num_epochs}'
)

In [36]:
# params = list(model.named_parameters())
# list(model.named_parameters())

In [37]:
# embs = model.embedding.weight

In [38]:
# npembs = embs.detach().to('cpu').numpy()

In [39]:
# npembs

In [40]:
# npembs[:,0]

In [41]:
# npembs[:,1]

In [42]:
# plt.scatter(npembs[:,0], npembs[:,1])
# r=0
# for label,x,y in zip(index_to_w, npembs[:,0], npembs[:,1]):
#     plt.annotate(label, xy=(x,y), xytext=(-1,1), textcoords='offset points', ha='center', va='bottom')
#     r+=1
# plt.show()

In [43]:
# index_to_w

In [44]:
# params[0][1]